In this notebook I'm gonna try Doc2Vec; a document embeddding technique for feature representation in NLP and I will use it for text classification

In [1]:
# first lets do some imports
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import nltk 
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import Doc2Vec, TaggedDocument